In [1]:
from pathlib import Path
from flask import Flask, request, abort,send_from_directory
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration, 
    ApiClient, 
    MessagingApi, 
    ReplyMessageRequest, 
    TextMessage,
    ImageMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent
)
from line_token import *
import requests

config_file = Path("line_token.py")

app = Flask(__name__)
line_bot_api = Configuration(access_token=CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# ngrok轉發網址
prefix_url = "https://7ab1-220-128-132-194.ngrok-free.app"

@app.route('/')
def hello():
    return 'Hello, World!'

@app.route('/files/<path>')
def get_tmp_path(path):
    return send_from_directory('files', path)

@app.route('/callback',methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    try:
        handler.handle(body, signature)
        return "OK"
    except InvalidSignatureError as e:
        print(e)
        abort(400)

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event: MessageEvent):
    with ApiClient(line_bot_api) as api_client:
        message = MessagingApi(api_client)
        reply_token = event.reply_token
        replyText = event.message.text
        print(f"client submit {replyText}")

        list_reply = [
            TextMessage(text=replyText)
        ]

        message.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=reply_token,
                messages=list_reply
            )
        )

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_message(event: MessageEvent):
    with ApiClient(line_bot_api) as api_client:
        message = MessagingApi(api_client)
        reply_token = event.reply_token
        image_id = event.message.id
        print(f"client submit a image")

        my_headers = {
            'Authorization': 'Bearer ' + CHANNEL_ACCESS_TOKEN
        }
        url = f"https://api-data.line.me/v2/bot/message/{image_id}/content"

        response = requests.get(url, headers=my_headers)
        with open(f'files/{image_id}.jpg', 'wb') as f:
            f.write(response.content)

        reply_content = f"已收到圖片: {image_id}.jpg"

        list_reply = [
            TextMessage(text=reply_content),
            ImageMessage(original_content_url=f"{prefix_url}/files/{image_id}.jpg", preview_image_url=f"{prefix_url}/files/{image_id}.jpg")
        ]

        message.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=reply_token,
                messages=list_reply
            )
        )

In [2]:
app.run(host='0.0.0.0', port=5000, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.109:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Aug/2024 13:38:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2024 13:38:49] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Aug/2024 13:39:17] "GET /files/520261098592272835.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2024 15:06:42] "GET /files/520261098592272835.jpg HTTP/1.1" 304 -
127.0.0.1 - - [12/Aug/2024 15:06:46] "GET /files/520261098592272835.jpg HTTP/1.1" 304 -
